In [2]:
# %pip install python-dotenv
# %pip install gTTS
# %pip install pyttsx3
# %pip install openai
# # Install pytorch for your machine
# %pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
# %pip install pydub
# To get pydub to work, may need to   sudo apt install ffmpeg
# %pip install transformers
# %pip install xformers

In [1]:
import os
from dotenv import load_dotenv
import pyttsx3
from gtts import gTTS
# from playsound import playsound
import openai
import torch
import numpy as np
import math
import time
from pydub import AudioSegment
# import transformers
# from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig
import re

/home/petern/miniconda3/envs/med_ai/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
# Not Currently in Use Yet
# load_dotenv()
# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# openai.api_key = OPENAI_API_KEY
def chatgbt(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        prompt=prompt,
        max_tokens=100
    )
    # message = response.choices[0].text.strip()
    return response

In [5]:
# Just reads meditation scripts from .txt file for now
FILE_NAME = "body-scan-gpt.txt"  # Might incorporate file name as cmd line argument later
SCRIPTS_DIR_NAME = "meditation_scripts"

def get_text():
    """Read text from the indicated .txt file"""
    curr_path = os.getcwd()
    dir_path = os.path.join(curr_path, SCRIPTS_DIR_NAME)
    file_path = os.path.join(dir_path, FILE_NAME)
    with open(file_path, "r") as f:
        unparsed_text = f.read()

    # Remove the text without brackets
    meditation_script = re.sub(r'\[.*?\]', '', unparsed_text)
    
    # Need to clean out the brief bracketed pauses
    return meditation_script

In [6]:
GTTS_DIR_NAME = 'gtts-voices'

def text_to_speech(meditation_script):
    # gTTS implementation

    # List of accents available
    accents = ['com.au', 'co.uk', 'us', 'ca', 'co.in', 'ie', 'co.za']
    speech_file_names = []

    for accent in accents:
        speech_file_name = f"gtts-{accent}.mp3"

        # Add longer pause after every period
        segments = meditation_script.split('. ')
        pause = AudioSegment.silent(duration=2000)  # milliseconds
        speech_audio = AudioSegment.empty()
        for segment in segments:
            # Create the TTS object
            tts = gTTS(segment,
                    tld=accent,    # Indian voice sounded the most soothing
                    lang="en",
                    slow=True
                    )
            tts.save('segment.mp3')
            segment_audio = AudioSegment.from_mp3('segment.mp3')
            speech_audio += segment_audio + pause

        # Save the audio file
        print(f"Saving {speech_file_name} audio file")
        path = os.path.join(GTTS_DIR_NAME, speech_file_name)
        speech_audio.export(path, format='mp3')

        speech_file_names.append(speech_file_name)

        os.remove('segment.mp3')  # Remove the temporary file
        
    return speech_file_names

In [7]:
PYTTSX3_DIR_NAME = 'pyttsx3-voices'

def text_to_speech_pyttsx3(meditation_script):
    # pyttsx3 implementation

    engine = pyttsx3.init() # object creation
    engine.setProperty('voice', 'en-us')
    rate = engine.getProperty('rate')   # getting details of current speaking rate
    engine.setProperty('rate', 125)     # setting up new voice rate
    volume = engine.getProperty('volume')   # getting to know current volume level (min=0 and max=1)
    engine.setProperty('volume', 1.0)    # setting up volume level  between 0 and 1
    engine.setProperty('pitch', 0.8)

    # Listen to events ??

    # Listen to different voices (only 2 options??) 
    # voices = engine.getProperty("voices")
    # for voice in voices:
        # engine.setProperty("voice", voice.id)
    #     print(f"voice id {voice.id}")
    #     engine.say("Hello")
    # engine.runAndWait()

    # Save to audio file
    print("Saving pyttsx3 audio file")
    engine.save_to_file(meditation_script, os.path.join(PYTTSX3_DIR_NAME, 'pyttsx3.mp3'))
    engine.runAndWait()
    engine.stop()

In [20]:
GTTS_DIR_NAME = 'gtts-voices'

def overlay_music_and_speech(path_to_file, speech_file_name):
    """Add background music to speech"""
    
    music1 = AudioSegment.from_mp3("music-only1.mp3")
    speech = AudioSegment.from_mp3(os.path.join(path_to_file, speech_file_name))
    times_to_repeat = len(speech) / len(music1)
    # Lengthen music so it is at least the length of speech audio
    longer_music = AudioSegment.empty()
    for _ in range(math.ceil(times_to_repeat)):
        longer_music += music1

    combined = speech.overlay(longer_music)
    combined.export(f"full-{speech_file_name}", format="mp3")

In [12]:
# def main():
meditation_script = get_text()

In [13]:
speech_file_names = text_to_speech(meditation_script)

Saving gtts-com.au.mp3 audio file
Saving gtts-co.uk.mp3 audio file
Saving gtts-us.mp3 audio file
Saving gtts-ca.mp3 audio file
Saving gtts-co.in.mp3 audio file
Saving gtts-ie.mp3 audio file
Saving gtts-co.za.mp3 audio file


In [21]:
# Add music to background of speech
path_to_file = GTTS_DIR_NAME
overlay_music_and_speech(path_to_file, speech_file_names[4])  # for just the indian accent audio for now

Saving combined audio


In [ ]:
# if __name__ == "__main__":
#     main()